In [ ]:
# === ENSEMBLE MODEL PATHS ===
model_paths = [
    "/kaggle/input/saved-models/kaggle/working/saved_models/best_efficientnet_b0.pt",
    "/kaggle/input/saved-models/kaggle/working/saved_models/best_efficientnet_b2.pt",
    "/kaggle/input/saved-models/kaggle/working/saved_models/best_resnet34.pt",
    "/kaggle/input/saved-models/kaggle/working/saved_models/best_resnet50.pt",
    "/kaggle/input/saved-models/kaggle/working/saved_models/best_resnet101.pt"
    "/kaggle/input/saved-models/kaggle/working/saved_models/best_densenet121.pt",
    "/kaggle/input/saved-models/kaggle/working/saved_models/best_densenet201.pt",
    "/kaggle/input/saved-models/kaggle/working/saved_models/best_mobilenet_v3_large.pt",
    "/kaggle/input/saved-models/kaggle/working/saved_models/best_convnext_tiny.pt"
]

validation_accuracies = [
    0.9503, 0.9448, 0.9199,0.8950, 0.9199,0.9475,0.9448,0.9448,0.8204
    # corresponding accuracies
]


In [ ]:
# === Imports ===
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# === CONFIG ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 256
SAVE_DIR = "/kaggle/input/saved-models/kaggle/working/saved_models"

# TEST_IMG_DIR = "/kaggle/input/smai-project-dataset/Phase_2_data/images_train/images_train"
# TEST_CSV = "/kaggle/input/smai-project-dataset/Phase_2_data/labels_train.csv"

TEST_IMG_DIR = "/kaggle/input/test-data-rid/Images_Sub"
TEST_CSV = "/kaggle/input/test-data-rid/combined_filenames.csv"



# === TRANSFORM ===
test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# === MODEL LOADING ===
def get_model(model_name, num_classes):
    if model_name == "efficientnet_b0":
        model = models.efficientnet_b0(pretrained=False)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif model_name == "efficientnet_b2":
        model = models.efficientnet_b2(pretrained=False)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif model_name == "resnet34":
        model = models.resnet34(pretrained=False)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == "resnet50":
        model = models.resnet50(pretrained=False)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == "resnet101":
        model = models.resnet101(pretrained=False)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == "densenet121":
        model = models.densenet121(pretrained=False)
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    elif model_name == "densenet201":
        model = models.densenet201(pretrained=False)
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    elif model_name == "mobilenet_v3_large":
        model = models.mobilenet_v3_large(pretrained=False)
        model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
    elif model_name == "convnext_tiny":
        model = models.convnext_tiny(pretrained=False)
        model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes)
    else:
        raise ValueError(f"Model {model_name} not implemented.")
    return model

# List of models
model_names = [
    "efficientnet_b0", "efficientnet_b2", 
    "resnet34", "resnet50", "resnet101",
    "densenet121", "densenet201",
    "mobilenet_v3_large", "convnext_tiny"
]

NUM_CLASSES = 15

# Placeholder model accuracies (fill after validation)
model_accuracies = {
    "efficientnet_b0": 0.9503, 
    "efficientnet_b2": 0.9448, 
    "resnet34": 0.9199, 
    "resnet50": 0.8950, 
    "resnet101": 0.9199,
    "densenet121": 0.9475, 
    "densenet201": 0.9448,
    "mobilenet_v3_large": 0.9448, 
    "convnext_tiny": 0.8204
}

# Normalize accuracies into weights
acc_values = np.array(list(model_accuracies.values()))
model_weights = acc_values / acc_values.sum()

# Load all models
models_list = []
for model_name in model_names:
    model = get_model(model_name, num_classes=NUM_CLASSES)
    model.load_state_dict(torch.load(os.path.join(SAVE_DIR, f"best_{model_name}.pt"), map_location=DEVICE))
    model = model.to(DEVICE)
    model.eval()
    models_list.append(model)

print(f"✅ Loaded {len(models_list)} models for ensembling!")

# === INFERENCE ===
# Read test CSV
test_df = pd.read_csv(TEST_CSV)

# Prepare prediction storage
final_preds = []

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Inference"):
    img_path = os.path.join(TEST_IMG_DIR, row['filename'])
    image = Image.open(img_path).convert("RGB")
    image = test_transform(image).unsqueeze(0).to(DEVICE)

    outputs = torch.zeros((1, NUM_CLASSES), device=DEVICE)

    for model_idx, model in enumerate(models_list):
        with torch.no_grad():
            logits = model(image)
            outputs += model_weights[model_idx] * logits  # weighted sum

    pred = torch.argmax(outputs, dim=1).item()
    final_preds.append(pred + 1)

# Save predictions
test_df['Region_ID_pred'] = final_preds
test_df.to_csv("submission.csv", index=False)
print("🎯 Submission saved to submission.csv")